# Local-in-time rerun

In [2]:
import tools
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import copy
import time
%load_ext snakeviz
import os   
import pickle
import pprint
import lit 

In [ ]:
N = 512
L = 1.0
st = tools.ScalarTool(N,L)

th = np.sin(2*np.pi/L * st.X[0])
st.plot(th,high_quality=True)
plt.savefig('initial_condition.eps',format = 'eps',dpi = 500)
plt.show()

In [ ]:
N = 512
L = 1.0
vt = tools.VectorTool(N,L)
u = np.zeros((2,N,N))
u[0] = np.sin(2*np.pi/L * st.X[1])


vector = u
plt.rc('text', usetex=True)
plt.rc('font', family='serif', size=12)


m = max(round(vt.N / 18), 1)
Q = plt.quiver(vt.X[0][1:-1:m, 1:-1:m],
       vt.X[1][1:-1:m, 1:-1:m],
       vector[0][1:-1:m, 1:-1:m],
       vector[1][1:-1:m, 1:-1:m], linewidths=2.0)
plt.quiverkey(
Q, 0.8, 1.03, 2, r'%.2f $\frac{m}{s}$' % np.amax(vector), labelpos='E',)
plt.xlabel(r'$x$')
plt.ylabel(r'$y$')
plt.title('')
plt.xlim(0.0, vt.L)
plt.ylim(0.0, vt.L)
plt.axis('scaled')

plt.streamplot(vt.X[0][1:-1:m, 1:-1:m],
       vt.X[1][1:-1:m, 1:-1:m],
       vector[0][1:-1:m, 1:-1:m],
       vector[1][1:-1:m, 1:-1:m])
plt.savefig('velocity_field.eps',format = 'eps',dpi = 500)
plt.show()

In [3]:
N = 64
M = 1000
L = 1.0
h = L/N
T = 3.0
dt = T/M
kappa = 0.0
gamma = 1.0
ftype = np.float64
ctype = np.complex128

st = tools.ScalarTool(N,L)
vt = tools.VectorTool(N,L)


sol_lit = lit.sim(N=N,M=M-1,Pe = np.inf,plot=False,T=T,save_th_every=M-1,save_u_every=1)



dt =  0.003003003003003003
